### Initial Setting up

In [ ]:
# Dependencies
import json
import pandas as pd
import requests
from config import id_1, id_2

### Subsequent Run Only

In [ ]:
# Import csv and get the songs that's not in the set
song_df = pd.read_csv('../data/ochalady-sakusaku.csv')

In [ ]:
set_key = set(song_df['key'].tolist())
len(set_key)

### Data Extraction

In [ ]:
# List of Song, Date, URL, inviter
key = []
title = []
artist = []
date = []
inviter = []
song_url = []
recording_base_url = 'https://www.smule.com'

In [ ]:
# Building url
user_1 = f'https://www.smule.com/s/profile/performance/{id_1}/sing?offset='
user_2 = f'https://www.smule.com/s/profile/performance/{id_2}/sing?offset='
offset = 0

id_1 is the VIP

In [ ]:
# Infinite loop until the list is empty (offset ran out)

while True:
    response = requests.get(f'{user_2}{offset}')
    # Get out of the loop if request is unsuccessful
    if response.status_code != 200:
        print('Unsuccessful Request T^T')
        break
    response_json = response.json()
    json_stuff = response_json['list']
    
    # If the list is empty in the json data
    if not json_stuff:
        break
    
    for x in range(0,len(json_stuff)):
#         if json_stuff[x]['key'] not in set_key: # Add this line and indent for subsequent run only
        if json_stuff[x]['performed_by'] == id_1:
            key.append(json_stuff[x]['key'])
            inviter.append(json_stuff[x]['performed_by'])
            title.append(json_stuff[x]['title'])
            artist.append(json_stuff[x]['artist'])
            date.append(json_stuff[x]['created_at'])
            recording_url = recording_base_url + json_stuff[x]['web_url']
            song_url.append(recording_url)
            print(f'Processing record index #{offset + x}: {offset}, #{x}')

#         else: # Add this line and indent for subsequent run only
#             print('All data has already been extracted') # Add this line and indent for subsequent run only
#             break # Add this line and indent for subsequent run only
            
    offset = response_json['next_offset']
    
    if offset == -1:
        print('No more records to be processed')
        break

In [ ]:
# Creating dataframe and save the first data request into csv
date_created = []

for d in date:
    date_only = d.split('T')[0]
    date_created.append(date_only)

# Show the list of songs in dataframe for id_2
id2_songs_df = pd.DataFrame({'key': key,
                             'Date_Created': date_created,
                             'Title':title,
                             'Artist': artist,
                             'Invite_Spawner': inviter,
                             'URL': song_url})

# Save first result to csv
id2_songs_df.to_csv('../data/id2_songs_df.csv', encoding='utf_8_sig', index=False)
id2_songs_df

In [ ]:
# How many songs did id_2 join id_1 in total?
join_id1 = len(title)
join_id1

id_2 is the VIP

In [ ]:
# Run another infinite loop again - You might have to wait 30mins b/c you'll get 418 code :P

while True:
    response = requests.get(f'{user_1}{offset}')
    # Get out of the loop if request is unsuccessful
    if response.status_code != 200:
        print('Unsuccessful Request T^T')
        break
    response_json = response.json()
    json_stuff = response_json['list']
    
    # If the list is empty in the json data
    if not json_stuff:
        break
    
    for x in range(0,len(json_stuff)):
#         if json_stuff[x]['key'] not in set_key: # Add this line and indent for subsequent run only
        if json_stuff[x]['performed_by'] == id_2:
            key.append(json_stuff[x]['key'])
            inviter.append(json_stuff[x]['performed_by'])
            title.append(json_stuff[x]['title'])
            artist.append(json_stuff[x]['artist'])
            date.append(json_stuff[x]['created_at'])
            recording_url = recording_base_url + json_stuff[x]['web_url']
            song_url.append(recording_url)
            print(f'Processing record index #{offset + x}: {offset}, #{x}')
#         else: # Add this line and indent for subsequent run only
#             print('All data has already been extracted') # Add this line and indent for subsequent run only
#             break # Add this line and indent for subsequent run only
    offset = response_json['next_offset']
    
    if offset == -1:
        print('No more records to be processed')
        break

In [ ]:
# Creating dataframe and save the first data request into csv
date_created = []

for d in date:
    date_only = d.split('T')[0]
    date_created.append(date_only)

# Show the list of songs in dataframe for id_2
id1_songs_df = pd.DataFrame({'key': key,
                             'Date_Created': date_created,
                             'Title':title,
                             'Artist': artist,
                             'Invite_Spawner': inviter,
                             'URL': song_url})

# Save first result to csv
id1_songs_df.to_csv('../data/id1_songs.csv', encoding='utf_8_sig', index=False)
id1_songs_df

In [ ]:
# How many songs did id_1 join id_2 in total?
join_id2 = len(title)
join_id2

Merging the two csv files

In [ ]:
# Import id2_songs.csv
id2_songs_df = pd.read_csv('../data/id2_songs.csv')
id2_songs_df

In [ ]:
# Append the two dataframes
all_songs_df = id1_songs_df.append(id2_songs_df)

# Sort the df by date
all_songs_df = all_songs_df.sort_values('Date_Created', ascending=False)

# Reset index
all_songs_df.reset_index(inplace=True)

# Delete old index
del all_songs_df['index']

In [ ]:
# Final result
print(f'{id_1} joined {id_2} on {len(id1_songs_df)} songs!')
print(f'{id_2} joined {id_1} on {len(id2_songs_df)} songs!')
print(f'You guys have made {len(all_songs_df)} recordings together as of this date.')

all_songs_df

In [ ]:
# Save as a csv with encoding that allows foreign character
all_songs_df.to_csv('../data/ochalady-sakusaku.csv', encoding='utf_8_sig')

Only if both runs go smoothly, run the code below and skip above

### Clean up & Export

In [ ]:
# Let's clean up the date column
date_created = []

for d in date:
    date_only = d.split('T')[0]
    date_created.append(date_only)
    
# Show the list of songs in dataframe
all_songs_df = pd.DataFrame({'key': key,
                             'Date_Created': date_created,
                             'Title':title,
                             'Artist': artist,
                             'Invite_Spawner': inviter,
                             'URL': song_url})

# Sort the df by date
all_songs_df = all_songs_df.sort_values('Date Created')

# Reset index
all_songs_df.reset_index(inplace=True)

# Delete old index
del all_songs_df['index']

In [ ]:
# Ta-da!
print(f'{id_1} joined {id_2} on {join_id2} songs!')
print(f'{id_2} joined {id_1} on {join_id1} songs!')
print(f'You guys have made {len(songs)} recordings together as of this date.')

all_songs_df

In [ ]:
# Save as a csv with encoding that allows foreign character
all_songs_df.to_csv('ochalady-sakusaku.csv', encoding='utf_8_sig')